https://pygis.io/docs/d_access_osm.html

https://nominatim.org/


In [ ]:
!pip install osmnx
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 2.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import spacy
!python -m spacy download en_core_web_sm

2023-12-04 03:19:23.149260: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 03:19:23.149359: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 03:19:23.149410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 03:19:24.647545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


load spacy model. Using small model here but can use larger

In [ ]:
nlp = spacy.load("en_core_web_sm")


For testing, manually inputted unformatted location output from LLM

In [ ]:
strings = ["Africa, North America, LA, Kansas cheney reservoir watershed, etc.", "Mesoamerican Reef catchments in Mexico, Belize, Guatemala, and Honduras, and the Yangtze River in China.",
           "Greater Mekong, which encompass Cambodia, Laos, Myanmar, Thailand, Vietnam",
           "Mesoamerican Reef catchments in Mexico, Belize, Guatemala, and Honduras, and the Yangtze River in China. // also amur-heilong, great barrier reef, mekong river, rio grande/rio bravo, united states",
           "brazil, chihuahua in mexico, pakistan, kabula river in india, cape town in africa, nebraska in US",
           "California, Hawaii, Virginia",
           "luangwa river in Zambia, Ghana, myanmar, gauri mahlia community forest in nepal",
           "Mesoamerican Reef catchments in Mexico,  Yangtze River in China., danube river, canada, guatemala, great barrier reef",
           "Kathmandu in Nepal, Mongolia, pakistan, chon kyzyl-suu river basin, bhutan",
           "brazil, guatemala, kenya, madagascar, mexico, tanzania, uganda, zimbabwe",
           "Mainland China, multiple regions in China. From a list of 15 cities: Beijing, Shanghai, Guangzhou, Chengdu, Xiamen, Kunming, Fuzhou, Xi'an, Shenyang, Tianjin, Nanning, Chongqing",
           "Asia: Bhutan, India, Kyrgyzstan, Nepal, Mongolia, Pakistan, ",
           "United States: 9 states such as Washington, Minnesota, Virginia, etc.",
           "Lower Kafue River Basin in Zambezi, Zamba",
           "Mainland China, multiple regions in China. From a list of 15 cities: Beijing, Shanghai, Guangzhou, Chengdu, Xiamen, Kunming, Fuzhou, Xi'an, Shenyang, Tianjin, Nanning, Chongqing",
           "Mongolia, Nepal, Asia High Mountains",
           "Asia High Mountains: India, Pakistan, Nepal, Mongolia, and Kyrgyz regions",
           "Asia High Mountains: India, Pakistan, Nepal, Mongolia, and Kyrgyz regions",
           "Arctic Region: Bristol Bay, Arctic Ocean Coast, and Bering Strait"]

defining all methods.

In [ ]:
# import osmnx
import osmnx as ox
import geopandas as gpd
import pandas as pd
list_of_locations = []
areas = []

#takes in the unformatted location outputs, empty list for list_of_locations and areas
#calls get_areas and returns its output, which is the list of geodataframes
def string_to_gdfs(strings, list_of_locations, areas):
  for string in strings:
    list_of_locations = spacy_extractions(string, list_of_locations)
  return get_areas(list_of_locations, areas)


#takes in raw string locaations, and an empty list to store list of locations.
#returns a list where each entry is a single location
def spacy_extractions(string,locations):
  # Process the text with spaCy
  doc = nlp(string)
  # Extract locations
  locs = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
  for loc in locs:
    locations.append(loc)
  return locations


#takes in filled locations list and empty areas list
#returns a list of geodataframes
def get_areas(locations, areas):
  for area in locations:
    try:
      areas.append(ox.geocode_to_gdf(area))
    except TypeError as e:
      print(f"caught Typeerror: {e}")
      continue
  return areas

#given the list of geodataframes, concats into one gdf, converts to geojson, and saves as a file to the drive
def saveToFile(areas):
  merged_gdf = gpd.pd.concat(areas)
  merged_gdf.to_json()
  merged_gdf.to_file("/content/drive/MyDrive/WWF x DSS/map.geojson", driver="GeoJSON")


method call

In [ ]:
areas = string_to_gdfs(strings, list_of_locations, areas)

saving to drive

In [ ]:
saveToFile(areas)

ValueError: ignored